In [ ]:
import copy
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
from tqdm.auto import tqdm
import xarray as xr

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
n_ens = 1001

In [ ]:
pd.options.display.max_columns = 50
df_configs = pd.read_csv('../data/fair2.1-parameters/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
valid_all = list(df_configs.index)[:n_ens]
df_configs

In [ ]:
df_solar = pd.read_csv(
    "../data/forcing/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/forcing/volcanic_ERF_monthly_-950001-201912.csv"
)

In [ ]:
volcanic_forcing = np.zeros(551)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]

In [ ]:
solar_forcing=np.zeros(551)
solar_forcing_df = pd.read_csv('../data/forcing/solar_erf_timebounds.csv', index_col=0)
solar_forcing[:281] = solar_forcing_df['erf'].values[:281]
solar_forcing[271:281] = np.linspace(1, 0, 10) * solar_forcing[271:281]

In [ ]:
trend_shape = np.ones(551)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
df_methane = pd.read_csv(
    "../data/fair2.1-parameters/CH4_lifetime.csv",
    index_col=0,
)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2300, 1)
scenarios = ['ssp119', 'ssp534', 'COFFEE1.1 EN_NPi2020_400f_lowBECCS']
f.define_scenarios(scenarios)
species, properties = read_properties()
f.define_species(species, properties)
f.define_configs(list(valid_all))
f.allocate()

In [ ]:
coffee = pd.read_csv('../data/emissions/COFFEE1.1___EN_NPi2020_400f_lowBECCS.csv', index_col=0)
ssp119 = pd.read_csv('../data/emissions/ssp119.csv', index_col=0)
ssp534 = pd.read_csv('../data/emissions/ssp534.csv', index_col=0)

In [ ]:
# add in emissions
for specie in ssp119.index:
    f.emissions.loc[dict(scenario='ssp119', specie=specie)] = ssp119.loc[specie, :].interpolate().values[:-1][:, None]
for specie in ssp534.index:
    f.emissions.loc[dict(scenario='ssp534', specie=specie)] = ssp534.loc[specie, :].interpolate().values[:-1][:, None]
for specie in coffee.index:
    f.emissions.loc[dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', specie=specie)] = coffee.loc[specie, :].values[:-1][:, None]

In [ ]:
pl.plot(f.emissions.loc[dict(specie='NOx aviation', config=valid_all[0])])

In [ ]:
# # I slightly modified ssp119 and ssp534 from RCMIP post-2100 because the CO2 ramp downs were not consistent
# # both go to zero in 2150
# # but I think 2150 is too quick a phase out for scenarios that are genuinely overshoots and designed to have long
# # term cooling. So modify to ramp to zero in 2250.
pl.plot(f.emissions.loc[dict(specie='CO2 FFI', config=valid_all[0])])

In [ ]:
pl.plot(f.emissions.loc[dict(specie='CO2 AFOLU', config=valid_all[0])])

In [ ]:
pl.plot(f.emissions.loc[dict(specie='N2O', config=valid_all[0])])

In [ ]:
pl.plot(f.emissions.loc[dict(specie='OC', config=valid_all[0])])

In [ ]:
# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs.loc[valid_all, "scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs.loc[valid_all, "solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs.loc[valid_all, "solar_trend"].values.squeeze(),
    specie="Solar",
)

# climate response
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[valid_all, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[valid_all, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs.loc[valid_all,"epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs.loc[valid_all,"gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs.loc[valid_all,"sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs.loc[valid_all,"sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs.loc[valid_all,"seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[valid_all,"F_4xCO2"])

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs.loc[valid_all,"r0"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"], df_configs.loc[valid_all,"rA"].values.squeeze(), specie="CO2"
)
fill(f.species_configs["iirf_uptake"], df_configs.loc[valid_all,"rU"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_temperature"],
    df_configs.loc[valid_all,"rT"].values.squeeze(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[valid_all,"beta"].values.squeeze())
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[valid_all,"shape Sulfur"].values.squeeze(),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"], df_configs.loc[valid_all,"shape BC"].values.squeeze(), specie="BC"
)
fill(
    f.species_configs["aci_shape"], df_configs.loc[valid_all,"shape OC"].values.squeeze(), specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[valid_all,f"ari {specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[valid_all,f"scale {specie}"].values.squeeze(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[valid_all,"scale minorGHG"].values.squeeze(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[valid_all,f"o3 {specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs.loc[valid_all,"co2_concentration_1750"].values.squeeze(),
    specie="CO2",
)

initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

f.run()

In [ ]:
temp_norm_19952014 = np.ones(21)
temp_norm_19952014[0] = 0.5
temp_norm_19952014[-1] = 0.5

In [ ]:
# ssp119
pl.plot(f.temperature[:, 0, :, 0].values - np.average(f.temperature[245:266, 0, :, 0].values, axis=0, weights=temp_norm_19952014) + 0.85);

In [ ]:
# ssp534
pl.plot(f.temperature[:, 1, :, 0].values - np.average(f.temperature[245:266, 1, :, 0].values, axis=0, weights=temp_norm_19952014) + 0.85);

In [ ]:
# coffee
pl.plot(f.temperature[:, 2, :, 0].values - np.average(f.temperature[245:266, 2, :, 0].values, axis=0, weights=temp_norm_19952014) + 0.85);

In [ ]:
# first index of 1.5C exceedence. If zero, run stays under 1.5C
exceeds_1p5_indices = np.argmax(
    f.temperature[:, :, :, 0].values - 
    np.average(f.temperature[245:266, 0, :, 0].values, axis=0, weights=temp_norm_19952014) + 0.85 > 1.5,
    axis=0
)
exceeds_1p5_indices

In [ ]:
# how many ensemble members exceed 1.5?
print(exceeds_1p5_indices[0, exceeds_1p5_indices[0, :]>0].shape)
print(exceeds_1p5_indices[1, exceeds_1p5_indices[1, :]>0].shape)
print(exceeds_1p5_indices[2, exceeds_1p5_indices[2, :]>0].shape)

In [ ]:
#np.min(exceeds_1p5_indices[0, exceeds_1p5_indices[0, :]>0])
#np.min(exceeds_1p5_indices[1, exceeds_1p5_indices[1, :]>0])
#np.min(exceeds_1p5_indices[2, exceeds_1p5_indices[2, :]>0])
exceeds_1p5_indices[2, exceeds_1p5_indices[2, :]>0].shape

In [ ]:
# does the config exceed 1.5C?
run_exceeds_1p5 = np.zeros_like(exceeds_1p5_indices, dtype=bool)
run_exceeds_1p5[exceeds_1p5_indices > 0] = True
run_exceeds_1p5

In [ ]:
np.sum(run_exceeds_1p5)

In [ ]:
f_fut = {}

In [ ]:
f_fut[2015] = FAIR(ch4_method="Thornhill2021")

In [ ]:
f_fut[2015].define_time(1750, 2016, 1)

In [ ]:
f_fut[2015].define_scenarios(scenarios)

In [ ]:
species, properties = read_properties()

In [ ]:
f_fut[2015].define_species(species, properties)

In [ ]:
f_fut[2015].define_configs(valid_all)

In [ ]:
f_fut[2015].configs

In [ ]:
f_fut[2015].allocate()

In [ ]:
#f_fut[2015].fill_from_rcmip()

# add in emissions
for specie in ssp119.index:
    f_fut[2015].emissions.loc[dict(scenario='ssp119', specie=specie)] = ssp119.loc[specie, :].interpolate().values[:266][:, None]
for specie in ssp534.index:
    f_fut[2015].emissions.loc[dict(scenario='ssp534', specie=specie)] = ssp534.loc[specie, :].interpolate().values[:266][:, None]
for specie in coffee.index:
    f_fut[2015].emissions.loc[dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', specie=specie)] = coffee.loc[specie, :].values[:266][:, None]

In [ ]:
# solar and volcanic forcing
fill(
    f_fut[2015].forcing,
    volcanic_forcing[:267, None, None] * df_configs.loc[valid_all, "scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f_fut[2015].forcing,
    solar_forcing[:267, None, None] * df_configs.loc[valid_all, "solar_amplitude"].values.squeeze()
    + trend_shape[:267, None, None] * df_configs.loc[valid_all, "solar_trend"].values.squeeze(),
    specie="Solar",
)

# climate response
fill(f_fut[2015].climate_configs["ocean_heat_capacity"], df_configs.loc[valid_all, "c1":"c3"].values)
fill(
    f_fut[2015].climate_configs["ocean_heat_transfer"],
    df_configs.loc[valid_all, "kappa1":"kappa3"].values,
)
fill(f_fut[2015].climate_configs["deep_ocean_efficacy"], df_configs.loc[valid_all, "epsilon"])
fill(f_fut[2015].climate_configs["gamma_autocorrelation"], df_configs.loc[valid_all, "gamma"])
fill(f_fut[2015].climate_configs["sigma_eta"], df_configs.loc[valid_all, "sigma_eta"])
fill(f_fut[2015].climate_configs["sigma_xi"], df_configs.loc[valid_all, "sigma_xi"])
fill(f_fut[2015].climate_configs["seed"], df_configs.loc[valid_all, "seed"])
fill(f_fut[2015].climate_configs["stochastic_run"], False)
fill(f_fut[2015].climate_configs["use_seed"], True)
fill(f_fut[2015].climate_configs["forcing_4co2"], df_configs.loc[valid_all, "F_4xCO2"])

# species level
f_fut[2015].fill_species_configs()

# carbon cycle
fill(f_fut[2015].species_configs["iirf_0"], df_configs.loc[valid_all,"r0"], specie="CO2")
fill(
    f_fut[2015].species_configs["iirf_airborne"], df_configs.loc[valid_all,"rA"], specie="CO2"
)
fill(f_fut[2015].species_configs["iirf_uptake"], df_configs.loc[valid_all,"rU"], specie="CO2")
fill(
    f_fut[2015].species_configs["iirf_temperature"],
    df_configs.loc[valid_all,"rT"],
    specie="CO2",
)

# aerosol indirect
fill(f_fut[2015].species_configs["aci_scale"], df_configs.loc[valid_all,"beta"])
fill(
    f_fut[2015].species_configs["aci_shape"],
    df_configs.loc[valid_all, "shape Sulfur"],
    specie="Sulfur",
)
fill(
    f_fut[2015].species_configs["aci_shape"], df_configs.loc[valid_all, "shape BC"], specie="BC"
)
fill(
    f_fut[2015].species_configs["aci_shape"], df_configs.loc[valid_all, "shape OC"], specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f_fut[2015].species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f_fut[2015].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f_fut[2015].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f_fut[2015].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f_fut[2015].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f_fut[2015].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f_fut[2015].species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f_fut[2015].species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f_fut[2015].species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f_fut[2015].species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f_fut[2015].species_configs["erfari_radiative_efficiency"],
        df_configs.loc[valid_all, f"ari {specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f_fut[2015].species_configs["forcing_scale"],
        df_configs.loc[valid_all, f"scale {specie}"],
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f_fut[2015].species_configs["forcing_scale"],
        df_configs.loc[valid_all,"scale minorGHG"],
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f_fut[2015].species_configs["ozone_radiative_efficiency"],
        df_configs.loc[valid_all, f"o3 {specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f_fut[2015].species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f_fut[2015].species_configs["baseline_concentration"],
    df_configs.loc[valid_all,"co2_concentration_1750"],
    specie="CO2",
)

initialise(f_fut[2015].concentration, f_fut[2015].species_configs['baseline_concentration'])
initialise(f_fut[2015].forcing, 0)
initialise(f_fut[2015].temperature, 0)
initialise(f_fut[2015].airborne_emissions, 0)
initialise(f_fut[2015].cumulative_emissions, 0)

f_fut[2015].run()

In [ ]:
pl.plot(f_fut[2015].temperature[:,0,:,0]);

In [ ]:
pl.plot(f_fut[2015].temperature[:,1,:,0]);

In [ ]:
pl.plot(f_fut[2015].temperature[:,2,:,0]);

In [ ]:
for year in tqdm(range(2016, 2300)):
    run = 0
    f_fut[year] = FAIR(ch4_method="Thornhill2021")
    f_fut[year].define_time(year, year+1, 1)
    f_fut[year].define_scenarios(scenarios)
    f_fut[year].define_species(species, properties)
    f_fut[year].define_configs(valid_all)
    f_fut[year].allocate()
    
    # emissions in
    for specie in ssp119.index:
        f_fut[year].emissions.loc[dict(scenario='ssp119', specie=specie)] = ssp119.loc[specie, :].interpolate().values[year-1750]
    for specie in ssp534.index:
        f_fut[year].emissions.loc[dict(scenario='ssp534', specie=specie)] = ssp534.loc[specie, :].interpolate().values[year-1750]
    for specie in coffee.index:
        f_fut[year].emissions.loc[dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', specie=specie)] = coffee.loc[specie, :].values[year-1750]
    
    # climate response
    fill(f_fut[year].climate_configs["ocean_heat_capacity"], df_configs.loc[valid_all, "c1":"c3"].values)
    fill(
        f_fut[year].climate_configs["ocean_heat_transfer"],
        df_configs.loc[valid_all, "kappa1":"kappa3"].values,
    )
    fill(f_fut[year].climate_configs["deep_ocean_efficacy"], df_configs.loc[valid_all, "epsilon"])
    fill(f_fut[year].climate_configs["gamma_autocorrelation"], df_configs.loc[valid_all, "gamma"])
    fill(f_fut[year].climate_configs["sigma_eta"], df_configs.loc[valid_all, "sigma_eta"])
    fill(f_fut[year].climate_configs["sigma_xi"], df_configs.loc[valid_all, "sigma_xi"])
    fill(f_fut[year].climate_configs["seed"], df_configs.loc[valid_all, "seed"])
    fill(f_fut[year].climate_configs["stochastic_run"], False)
    fill(f_fut[year].climate_configs["use_seed"], True)
    fill(f_fut[year].climate_configs["forcing_4co2"], df_configs.loc[valid_all, "F_4xCO2"])

    # species level
    f_fut[year].fill_species_configs()

    # carbon cycle
    fill(f_fut[year].species_configs["iirf_0"], df_configs.loc[valid_all,"r0"], specie="CO2")
    fill(
        f_fut[year].species_configs["iirf_airborne"], df_configs.loc[valid_all,"rA"], specie="CO2"
    )
    fill(f_fut[year].species_configs["iirf_uptake"], df_configs.loc[valid_all,"rU"], specie="CO2")
    fill(
        f_fut[year].species_configs["iirf_temperature"],
        df_configs.loc[valid_all,"rT"],
        specie="CO2",
    )

    # aerosol indirect
    fill(f_fut[year].species_configs["aci_scale"], df_configs.loc[valid_all,"beta"])
    fill(
        f_fut[year].species_configs["aci_shape"],
        df_configs.loc[valid_all, "shape Sulfur"],
        specie="Sulfur",
    )
    fill(
        f_fut[year].species_configs["aci_shape"], df_configs.loc[valid_all, "shape BC"], specie="BC"
    )
    fill(
        f_fut[year].species_configs["aci_shape"], df_configs.loc[valid_all, "shape OC"], specie="OC"
    )

    # methane lifetime baseline and sensitivity
    fill(
        f_fut[year].species_configs["unperturbed_lifetime"],
        df_methane.loc["historical_best", "base"],
        specie="CH4",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "CH4"],
        specie="CH4",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "N2O"],
        specie="N2O",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "VOC"],
        specie="VOC",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "NOx"],
        specie="NOx",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "HC"],
        specie="Equivalent effective stratospheric chlorine",
    )
    fill(
        f_fut[year].species_configs["lifetime_temperature_sensitivity"],
        df_methane.loc["historical_best", "temp"],
    )

    # emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
    # might wanna run pulse expts with these gases)
    fill(f_fut[year].species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
    fill(f_fut[year].species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
    fill(f_fut[year].species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

    # aerosol direct
    for specie in [
        "BC",
        "CH4",
        "N2O",
        "NH3",
        "NOx",
        "OC",
        "Sulfur",
        "VOC",
        "Equivalent effective stratospheric chlorine",
    ]:
        fill(
            f_fut[year].species_configs["erfari_radiative_efficiency"],
            df_configs.loc[valid_all, f"ari {specie}"],
            specie=specie,
        )

    # forcing scaling
    for specie in [
        "CO2",
        "CH4",
        "N2O",
        "Stratospheric water vapour",
        "Contrails",
        "Light absorbing particles on snow and ice",
        "Land use",
    ]:
        fill(
            f_fut[year].species_configs["forcing_scale"],
            df_configs.loc[valid_all, f"scale {specie}"],
            specie=specie,
        )

    for specie in [
        "CFC-11",
        "CFC-12",
        "CFC-113",
        "CFC-114",
        "CFC-115",
        "HCFC-22",
        "HCFC-141b",
        "HCFC-142b",
        "CCl4",
        "CHCl3",
        "CH2Cl2",
        "CH3Cl",
        "CH3CCl3",
        "CH3Br",
        "Halon-1211",
        "Halon-1301",
        "Halon-2402",
        "CF4",
        "C2F6",
        "C3F8",
        "c-C4F8",
        "C4F10",
        "C5F12",
        "C6F14",
        "C7F16",
        "C8F18",
        "NF3",
        "SF6",
        "SO2F2",
        "HFC-125",
        "HFC-134a",
        "HFC-143a",
        "HFC-152a",
        "HFC-227ea",
        "HFC-23",
        "HFC-236fa",
        "HFC-245fa",
        "HFC-32",
        "HFC-365mfc",
        "HFC-4310mee",
    ]:
        fill(
            f_fut[year].species_configs["forcing_scale"],
            df_configs.loc[valid_all,"scale minorGHG"],
            specie=specie,
        )

    # ozone
    for specie in [
        "CH4",
        "N2O",
        "Equivalent effective stratospheric chlorine",
        "CO",
        "VOC",
        "NOx",
    ]:
        fill(
            f_fut[year].species_configs["ozone_radiative_efficiency"],
            df_configs.loc[valid_all, f"o3 {specie}"],
            specie=specie,
        )

    # tune down volcanic efficacy
    fill(f_fut[year].species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


    # initial condition of CO2 concentration (but not baseline for forcing calculations)
    fill(
        f_fut[year].species_configs["baseline_concentration"],
        df_configs.loc[valid_all,"co2_concentration_1750"],
        specie="CO2",
    )
    
    while True:
        run = run+1
        initialise(f_fut[year].concentration, f_fut[year-1].concentration[-1, ...])
        initialise(f_fut[year].forcing, f_fut[year-1].forcing[-1, ...])
        initialise(f_fut[year].temperature, f_fut[year-1].temperature[-1, ...])
        initialise(f_fut[year].airborne_emissions, f_fut[year-1].airborne_emissions[-1, ...])
        initialise(f_fut[year].cumulative_emissions, f_fut[year-1].cumulative_emissions[-1, ...])
        initialise(f_fut[year].alpha_lifetime, f_fut[year-1].alpha_lifetime[-1, ...])
        f_fut[year].gas_partitions=copy.deepcopy(f_fut[year-1].gas_partitions)
        
        # Volcanic forcing
        fill(
            f_fut[year].forcing, 
            volcanic_forcing[year-1750:year-1749, None, None] * 
            df_configs.loc[valid_all, 'scale Volcanic'].values.squeeze(), 
            specie='Volcanic'
        )

        # Solar forcing
        fill(f_fut[year].forcing, 
             solar_forcing[year-1750:year-1749, None, None] * 
             df_configs.loc[valid_all, 'solar_amplitude'].values.squeeze() + 
             trend_shape[year-1750:year-1749, None, None] * df_configs.loc[valid_all, 'solar_trend'].values.squeeze(),
             specie='Solar'
        )

        # do the run
        f_fut[year].run(progress=False)
        
        # naive optimizer!
        t_anom = (f_fut[year].temperature[-1, :, :, 0] - np.average(f_fut[2015].temperature[245:266, :, :, 0], weights=temp_norm_19952014, axis=0)) + 0.85
        c_ch4 = f_fut[year].concentration[-1, :, :, 3]
        
        #print(f"temperature in {year} is {t_anom.values}")
        overshoot_configs = np.logical_and(t_anom>1.5, c_ch4>50)
        print(np.sum(overshoot_configs).values)
        
        # all below 1.5C? great; move on to next year, else continue
        if np.sum(overshoot_configs).values==0:# or run>=100:
            break
        
        shave = np.zeros_like(t_anom)
        #shave[overshoot_configs] = np.maximum(10, (t_anom.values[overshoot_configs]-1.5)*10000)
        shave[overshoot_configs] = 20
        f_fut[year].emissions[0, :, :, 3] = f_fut[year].emissions[0, :, :, 3] - shave

In [ ]:
concentration_opt = np.zeros((551, n_ens))
concentration_opt[:267] = f_fut[2015].concentration[:, 0, :, 3]
for year in range(2016, 2300):
    concentration_opt[year-1749] = f_fut[year].concentration[1, 0, :, 3]

In [ ]:
pl.plot(concentration_opt);
pl.ylim(0,2000);

In [ ]:
# for year in range(2015, 2300):
#     pl.plot(f_fut[year].timebounds, f_fut[year].temperature[:, 0, :, 0] - np.average(f_fut[2015].temperature[245:266, 0, :, 0], axis=0, weights=temp_norm_19952014) + 0.85, color='k')
# pl.plot(f.timebounds[250:], f.temperature[250:,0,:,0] - np.average(f.temperature[245:266, 0, :, 0], axis=0, weights=temp_norm_19952014) + 0.85, color='r')

In [ ]:
species[58]

In [ ]:
emissions_opt = np.zeros((550, 3, n_ens))
emissions_opt[:266] = f_fut[2015].emissions[:, :, :, 3]
for year in range(2016, 2300):
    emissions_opt[year-1750] = f_fut[year].emissions[:, :, :, 3]
    
concentration_opt = np.zeros((551, 3, n_ens))
concentration_opt[:267] = f_fut[2015].concentration[:, :, :, 3]
for year in range(2016, 2300):
    concentration_opt[year-1749] = f_fut[year].concentration[1, :, :, 3]

forcing_ch4_opt = np.zeros((551, 3, n_ens))
forcing_ch4_opt[:267] = f_fut[2015].forcing[:, :, :, 3]
for year in range(2016, 2300):
    forcing_ch4_opt[year-1749] = f_fut[year].forcing[1, :, :, 3]

forcing_o3_opt = np.zeros((551, 3, n_ens))
forcing_o3_opt[:267] = f_fut[2015].forcing[:, :, :, 58]
for year in range(2016, 2300):
    forcing_o3_opt[year-1749] = f_fut[year].forcing[1, :, :, 58]    
    
forcing_sum_opt = np.zeros((551, 3, n_ens))
forcing_sum_opt[:267] = f_fut[2015].forcing_sum
for year in range(2016, 2300):
    forcing_sum_opt[year-1749] = f_fut[year].forcing_sum[1, :, :]
    
temperature_opt = np.zeros((551, 3, n_ens))
temperature_opt[:267] = f_fut[2015].temperature[:, :, :, 0]
for year in range(2016, 2300):
    temperature_opt[year-1749] = f_fut[year].temperature[1, :, :, 0]
    
lifetime_opt = np.zeros((551, 3, n_ens))
lifetime_opt[:267] = f_fut[2015].alpha_lifetime[:, :, :, 3] * df_methane.loc["historical_best", "base"]
for year in range(2016, 2300):
    lifetime_opt[year-1749] = f_fut[year].alpha_lifetime[0, :, :, 3] * df_methane.loc["historical_best", "base"]

In [ ]:
ds_opt = xr.Dataset(
    data_vars = dict(
        emissions = (["timepoint", "scenario", "config"], emissions_opt),
        concentration = (["timebound", "scenario", "config"], concentration_opt),
        forcing_ch4 = (["timebound", "scenario", "config"], forcing_ch4_opt),
        forcing_o3 = (["timebound", "scenario", "config"], forcing_o3_opt),
        forcing_sum = (["timebound", "scenario", "config"], forcing_sum_opt),
        temperature = (["timebound", "scenario", "config"], temperature_opt),
        lifetime = (["timebound", "scenario", "config"], lifetime_opt),
    ),
    coords = dict(
        timepoint=np.arange(1750.5, 2300),
        timebound=np.arange(1750, 2301),
        scenario=scenarios,
        config=valid_all
    )
)

In [ ]:
ds_opt

In [ ]:
os.makedirs('../results', exist_ok=True)

In [ ]:
f.to_netcdf('../results/noCH4R.nc')

In [ ]:
ds_opt.to_netcdf('../results/CH4R.nc')

In [ ]:
(f.alpha_lifetime[:, :, :, 3] * df_methane.loc["historical_best", "base"])

In [ ]:
# Save lifetime
(f.alpha_lifetime[:, :, :, 3] * df_methane.loc["historical_best", "base"]).to_netcdf('../results/noCH4R_lifetime.nc')

In [ ]:
# NEED TO GATHER INTO A DATASET

# pd.DataFrame(concentration_opt, index=f.timebounds, columns=valid_all).to_csv('../results/ssp119_CH4R_concentrations.csv')
# pd.DataFrame(emissions_opt, index=f.timepoints, columns=valid_all).to_csv('../results/ssp119_CH4R_emissions.csv')
# pd.DataFrame(forcing_opt, index=f.timebounds, columns=valid_all).to_csv('../results/ssp119_CH4R_forcing.csv')
# pd.DataFrame(temperature_opt, index=f.timebounds, columns=valid_all).to_csv('../results/ssp119_CH4R_temperature.csv')
# pd.DataFrame(forcing_sum_opt, index=f.timebounds, columns=valid_all).to_csv('../results/ssp119_CH4R_forcing_sum.csv')
# pd.DataFrame(lifetime_opt, index=f.timebounds, columns=valid_all).to_csv('../results/ssp119_CH4R_lifetime.csv')
pd.DataFrame(np.array([f.ebms.ecs, f.ebms.tcr]).T, index=valid_all, columns=['ECS', 'TCR']).to_csv('../results/ecs_tcr.csv')